# Introduction
This notebook explores how false publications in science can emerge as the product of a multi-generational simulation of science.

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy
from scipy.stats import beta, binom, entropy
import random
import json
import copy
import math

# my modules
import scientist
import evaluation
import helper
import settings
import publisher

# global variables
num_bins = 4
num_draws = 10
num_participants = 10
num_generations = 3

In [2]:
# distribution of bins
bins_to_probs = {}
for i in range(0, num_bins):
    bins_to_probs[i] = 0.5

## Initialize participants

In [3]:
def make_participants(setting, alpha_value):
    participants = []

    for i in range (0, num_participants):
        if setting == "rate":
            report_set = settings.ReportingSetting("rate")
        elif setting == "data":
            report_set = settings.ReportingSetting("data")
        elif setting == "subset":
            report_set = settings.ReportingSetting("subset")

        # make participant
        participant = scientist.Participant(alpha=alpha_value, reporting_setting=report_set)
                        
        participants.append(participant)

    return(participants)

## Run an experiment

The multi-generational experiment is run, given reporting setting and exaggeration values.

In [4]:
def run_experiment(setting, alpha_value):
    # each experiment starts with a blank cannon (starts with 1-1 prior)
    scientific_record = {}
    for bin_num in range(0, num_bins):
        scientific_record[bin_num] = {} 
        scientific_record[bin_num][0] = 1
        scientific_record[bin_num][1] = 1
    
    for generation in range(0, num_generations):
        print(f"\n* Generation {generation}...")
        helper.print_record(scientific_record, num_bins)
        print(f"   Arm score: {evaluation.arm_parameter_score(scientific_record, bins_to_probs)}")
        print(f"   Entropy score: {evaluation.total_entropy_score(scientific_record, bins_to_probs)}")
        
        # each generation gets an entirely new set of participants
        participants = make_participants(setting, alpha_value)

        # scientists explore and submit reports
        for participant in participants:
            # sample
            for i in range(0, num_draws):
                bin_number, value = participant.sample(scientific_record, num_bins, bins_to_probs)
                
#                 print(f"   sample from bin {bin_number}: {value}")

            # choose the bin
            bin_choice = participant.choose_bin(scientific_record, num_bins, num_draws)
#             print(f"   chose bin {bin_choice}")

            # make a report
            participant.report(num_bins, num_draws)
            
        # the peer review board selects reports for publication and returns the updated scientific record
        scientific_record = publisher.peer_review(participants, scientific_record)
        
    print("\n\n* FINAL RESULTS")
    # final metric of how well scientists play the multi-armed bandit game
    print(evaluation.arm_parameter_score(scientific_record, bins_to_probs))

    # final metric of how well scientists reduce the entropy of the scientific record
    print(evaluation.total_entropy_score(scientific_record, bins_to_probs))

In [5]:
run_experiment("data", 0)


* Generation 0...
Scientific record
   bin 0: 1 zero(s), 1 one(s)
   bin 1: 1 zero(s), 1 one(s)
   bin 2: 1 zero(s), 1 one(s)
   bin 3: 1 zero(s), 1 one(s)

   Arm score: 0.0
   Entropy score: 0.0

* Generation 1...
Scientific record
   bin 0: 4 zero(s), 12 one(s)
   bin 1: 1 zero(s), 1 one(s)
   bin 2: 1 zero(s), 1 one(s)
   bin 3: 1 zero(s), 1 one(s)

   Arm score: 0.0625
   Entropy score: 0.035960259056472606

* Generation 2...
Scientific record
   bin 0: 4 zero(s), 12 one(s)
   bin 1: 3 zero(s), 7 one(s)
   bin 2: 6 zero(s), 2 one(s)
   bin 3: 1 zero(s), 1 one(s)

   Arm score: 0.175
   Entropy score: 0.09371469150604245


* FINAL RESULTS
0.1977272727272727
0.09791676785591555
